In [1]:
from tools.lib.logreader import MultiLogIterator, LogReader
import os
from selfdrive.car.gm.values import DBC, CAR, AccState, CanBus, STEER_THRESHOLD
from opendbc.can.parser import CANParser
from pprint import pprint as pp
import cereal.messaging as messaging
import unittest
import random
import multiprocessing

In [2]:
signals = [
    ("AcceleratorPedal2", "AcceleratorPedal2"),
    ("AcceleratorPedalChecksum", "AcceleratorPedal2"), 
    ("AcceleratorPedalPressed", "AcceleratorPedal2"),
    ("AccelPressedCruiseNotBraking", "AcceleratorPedal2"),
    ("Count1", "AcceleratorPedal2"),
    ("Count2", "AcceleratorPedal2"),
    ("Count3", "AcceleratorPedal2"),
    ("Countdown1", "AcceleratorPedal2"),
    ("Countdown2", "AcceleratorPedal2"),
    ("CruiseState", "AcceleratorPedal2"),
    ("CruiseToActive", "AcceleratorPedal2"),
    ("CruiseTransition", "AcceleratorPedal2"),
    ("DistanceToStop", "AcceleratorPedal2"),
    ("Set0_a", "AcceleratorPedal2"),
    ("Set0_b", "AcceleratorPedal2"),
    ("Set2", "AcceleratorPedal2"),
    ("Set24", "AcceleratorPedal2"),
    ("Set3", "AcceleratorPedal2"),
]
checks = [
    ("AcceleratorPedal2", 0),
]

In [3]:
path = '/home/user/data/rlogs/'
rlogs = [path + fn for fn in os.listdir(path) if fn[-4:] == '.bz2']

In [5]:
addresses = [452]
logs = rlogs[1:100]

In [8]:
# Collect can messages of interest
def can_from_rlog(rlog, addresses):
    # print(rlog)
    ret = []
    for msg in LogReader(rlog):
        if msg.which() == 'can':
            for can in msg.can:
                if can.address in addresses:
                    ret.append(can)
    return ret

can_msgs = []
with multiprocessing.Pool(16) as pool:
    for d in pool.starmap(can_from_rlog, [(rlog, addresses) for rlog in rlogs]):
        can_msgs.extend(d)
        print(len(can_msgs))

2347
4748
7147
9549
11949
14360
16752
19158
21558
23959
26359
28760
29055
31455
33856
36256
38646
41046
43446
45845
48245
50643
53043
55442
57841
60240
62640
65039
67444
69844
72243
74642
76190
78591
80993
82251
84648
87134
89535
91929
94329
96729
99129
101529
103929
106326
106532
109013
111414
113820
114782
117175
119579
121981
124385
126784
129184
131586
133981
136382
138782
141181
143581
145971
148371
150771
153173
155582
157983
160383
162795
165195
167596
169998
172401
174790
177184
179573
181973
184379
186780
189179
191579
193989
196389
198872
201272
203682
206082
208483
210883
213281
215681
218081
220481
222880
225288
227696
230096
230096


In [ ]:
class TestAcceleratorPedal2(unittest.TestCase):
    notbraking_invalid = 0
    notbraking_total = 0
    
    def values(self, v):
        checksum = (0x200 - int(v['AcceleratorPedal2']) - int(v['Count1'])) & 0x1FF
        self.assertEqual(v['AcceleratorPedalChecksum'], checksum)
        
        if v['AcceleratorPedalPressed']:
            self.assertGreater(v['AcceleratorPedal2'], 0)
        
        if (v['AcceleratorPedalPressed'] and (v['CruiseState'] != 0)) != bool(v['AccelPressedCruiseNotBraking']):
            self.notbraking_invalid += 1
        self.notbraking_total += 1

        self.assertEqual(v['Count1'], v['Count2']) 
        self.assertEqual(v['Count1'], v['Count3'])
        # self.assertEqual(v['Count1'] + v['Countdown1'], 3)
        # self.assertEqual(v['Countdown1'], v['Countdown2']) # mismatches all the time

        self.assertIn(v['CruiseState'], [0,1,3,4])
        self.assertLess(v['DistanceToStop'], 2000)
        self.assertIn(v['CruiseToActive'], [0,1])
        self.assertIn(v['CruiseTransition'], [0,1])
        
        self.assertEqual(v['Set0_a'], 0)
        self.assertEqual(v['Set0_b'], 0)
        self.assertEqual(v['Set2'], 2)
        self.assertEqual(v['Set24'], 24)
        self.assertIn(v['Set3'], [3])

In [ ]:
stop = False
can_parser = CANParser(DBC[CAR.VOLT]["pt"], signals, checks, CanBus.POWERTRAIN, enforce_checks=False)
test = TestAcceleratorPedal2()

for rlog in random.sample(rlogs, len(rlogs)):
    print(rlog)
    try:
        reader = MultiLogIterator([rlog])
    except:
        print(f'bad rlog: {rlog}')
        continue

    msgs = sorted(reader, key=lambda msg: msg.logMonoTime)
    for msg in msgs:
        if msg.which() == 'can':
            for can_msg in msg.can:
                if can_msg.address in [452]:
                    can_parser.update_string(msg.as_builder().to_bytes())
                    v = can_parser.vl['AcceleratorPedal2']
                    test.values(v)